<a href="https://colab.research.google.com/github/Sheraz55/Deep-learning-course-practice/blob/main/CH_08_Text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
import numpy as np

path = keras.utils.data_utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

614400/600901 [==============================] - 0s 0us/step
Corpus length: 600893


In [ ]:
maxlen = 60
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
  sentences.append(text[i: i + maxlen])
  next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57
Vectorization...


In [ ]:
from keras import layers
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [ ]:
from tensorflow.keras import optimizers
import tensorflow as tf


In [ ]:

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [ ]:
import random
import sys
for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    for temperature in [0.2, 0.5, 1.0, 1.2]:
      print('------ temperature:', temperature)
      sys.stdout.write(generated_text)

      for i in range(400):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(generated_text):
          sampled[0, t, char_indices[char]] = 1.
          preds = model.predict(sampled, verbose=0)[0]
          next_index = sample(preds, temperature)
          next_char = chars[next_index]
          generated_text += next_char
          generated_text = generated_text[1:]
          sys.stdout.write(next_char)

epoch 1
1565/1565 [==============================] - 32s 17ms/step - loss: 2.5225
--- Generating with seed: "ill, one might say arbitrary will, is now necessary for a
ge"
------ temperature: 0.2
ill, one might say arbitrary will, is now necessary for a
gettss e         et t  i  e s e ese e  s t tet s ta   e u  eer   tee e s it  e  te est eo tt  e st     t    s   eeaee       t  e  t e    t    t ss   se   t t  e     eeee  a atea     eeti  e es  s t t   e    st iettiie  r    t e  ei oc eii      te e   ttst tte ti  teee  ea  stet   e ta     tie eee    h  eietst   es eese  s    e  t  t e t    tt      e             ts   te    t e  t    e e ttt  e e e  st   e ea eae            saats  t  i s     e ttt e   it     e tee eiaee e t          e  ta      e    s  tese tsets  a te ts ite      e        eee eee     at       ei      s  s  st    e  s      e       e  t  e etrae te e         s   t t  et  tet    i aeoe       te  e tt  te  se e s  a  e  s   it t       e s     isa       s     t a teee  ts  t e 